In [232]:
from src.utils import load_dataset, get_path
from src.adapter import adapt_to_dataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [233]:
data_train_exaction = load_dataset(get_path('data', 'npy', 'audio_data_train.npy'))
data_test_exaction = load_dataset(get_path('data', 'npy', 'audio_data_test.npy'))

In [234]:
data_train = pd.read_csv('data/metadata_train_challenge.csv')
data_test = pd.read_csv('data/metadata_public_test.csv')
data_train = adapt_to_dataset(data_train, data_train_exaction)
data_test = adapt_to_dataset(data_test, data_test_exaction)

In [235]:
def parse_audio(x):
    return x.flatten('F')[:x.shape[0]] 

In [236]:
X_train = [i.audio_data.features.mfccs for i in data_train.samples]
X_train = [parse_audio(x) for x in X_train]
y_train = [i.assessment_result for i in data_train.samples]

X_test = [i.audio_data.features.mfccs for i in data_test.samples]
X_test = [parse_audio(x) for x in X_test]
y_test = [i.assessment_result for i in data_test.samples]


In [237]:
#X_train, x_test, Y_train, y_test =  train_test_split(X, Y, test_size=0.2)

In [243]:
# scaler = StandardScaler()
# scaler.fit(X_train)
# x_train_scaled = scaler.transform(X_train)
# x_test_scaled = scaler.transform(X_test)

# pca = PCA().fit(x_train_scaled)

# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# plt.xlabel('Number of Components')
# plt.ylabel('Variance (%)')
# plt.show()

In [239]:
grid_params = {
    'n_neighbors': [3, 5, 7, 9, 11, 15, 17, 40],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

model = GridSearchCV(KNeighborsClassifier(), grid_params, cv=5, n_jobs=-1)
model.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 7, 9, 11, 15, 17, 40],
                         'weights': ['uniform', 'distance']})

In [240]:
y_predict = model.predict(X_test)
acc = accuracy_score(y_test, y_predict)
acc

0.8742857142857143

In [241]:
fpr, tpr, threshold = roc_curve(y_test, y_predict)
roc_auc = auc(fpr, tpr)
roc_auc


C:\Users\Hieu\anaconda3\envs\covid\lib\site-packages\sklearn\metrics\_ranking.py:949: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn("No positive samples in y_true, "


nan

In [242]:
# from sklearn.metrics import roc_auc_score
# roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])